# 「行列計算アルゴリズム 第4章 線型方程式」用ノートブック

# ■メモ

### 概要
- 第4章「線形方程式」で使用するノートブックです。
- 各セルを順に実行することで、本書に記載のとおり実行結果が出力されます。
- 動作確認はJulia 1.10.0 で行っています。

### 使用するスクリプト（.jl）ファイル
- MCA_linsolve_lu.jl
- MCA_linsolve_jacobi.jl
- MCA_linsolve_gauss_seidel.jl
- MCA_linsolve_sor.jl
- MCA_linsolve_cg.jl
- MCA_lu_pivoting.jl（第3章で作成したもの）

### 事前にインストールが必要なパッケージ
- Krylov

# ■プログラム

## 4.2 直接法

### 4.2.2 LU分解に基づく計算法

In [ ]:
# 第3章で作成した MCA_lu_pivoting.jl を読み込む
dir = readdir("../", join=true)
include(dir[3]*"/MCA_lu_pivoting.jl")
include("MCA_linsolve_lu.jl");

In [ ]:
using Random
Random.seed!(1234)
n = 100; A = randn(n,n); b = randn(n);

In [ ]:
x = MCA_linsolve_lu(A,b)
using LinearAlgebra
println("relative residual norm = ", norm(b - A * x) / norm(b))

### 4.2.3 Juliaの関数

In [ ]:
x = A \ b　    # Ax = b の求解
println("relative residual norm = ", norm(b - A * x) / norm(b))

In [ ]:
F = lu(A); x = F.U \ (F.L \ b[F.p])    # 明示的にLU分解する場合
println("relative residual norm = ", norm(b - A * x) / norm(b))
# LinearAlgebraパッケージのluではこのような書き方も可
x = F \ b
println("relative residual norm = ", norm(b - A * x) / norm(b))

In [ ]:
using SparseArrays
Random.seed!(1234)
n = 100; A = sprandn(n,n,0.2)
x = A \ b
println("relative residual norm = ", norm(b - A * x) / norm(b))
F = lu(A); x = F \ b
println("relative residual norm = ", norm(b - A * x) / norm(b))

### 4.2.4 関連技術

In [ ]:
Random.seed!(1234)
n = 1000; A = randn(n,n); b = randn(n)
F_f32 = lu(Matrix{Float32}(A))   # 単精度計算を利用した低精度のLU分解
x = zeros(n); r = b
for k = 1:4
    r_f32 = Vector{Float32}(r)
    e_f32 = - (F_f32 \ r_f32)    # 単精度LU分解を利用した誤差方程式の求解
    x = x - e_f32                # 解の更新  （ここは倍精度で計算している）
    r = b - A * x                # 残差の計算（ここは倍精度で計算している）
    println("Iter = ", k, ", relative residual norm = ", norm(r)/norm(b))
end

## 4.3 定常反復法

### 4.3.2 ヤコビ法，ガウス・ザイデル法，SOR法

In [ ]:
include("MCA_linsolve_jacobi.jl")
include("MCA_linsolve_gauss_seidel.jl")
include("MCA_linsolve_sor.jl");

In [ ]:
n = 10; A = diagm(0 => 2*ones(n), 1 => -ones(n-1), -1 => -ones(n-1))
b = ones(n);

In [ ]:
x = MCA_linsolve_jacobi(A,b,iter=1000)
println("Jacobi      : ", norm(b - A*x) / norm(b))
x = MCA_linsolve_gauss_seidel(A,b,iter=1000)
println("Gauss-Seidel: ", norm(b - A*x) / norm(b))
x = MCA_linsolve_sor(A,b,omega=1.5,iter=1000)
println("SOR         : ", norm(b - A*x) / norm(b))

## 4.4 共役勾配法

### 4.4.4 具体的な実装法

In [ ]:
include("MCA_linsolve_cg.jl");

In [ ]:
n = 10; A = diagm(0 => 2*ones(n), 1 => -ones(n-1), -1 => -ones(n-1))
b = ones(n);

In [ ]:
x, r = MCA_linsolve_cg(A,b)
println("number of iterations   = ", size(r,1)-1)
println("relative residual norm = ", norm(b - A*x) / norm(b))

In [ ]:
using SparseArrays
n = 100; A = spdiagm(0 => range(2.01,3,n), 1 => ones(n-1), -1 => ones(n-1))
b = ones(n);

In [ ]:
x, r = MCA_linsolve_cg(A,b)
println("number of iterations   = ", size(r,1)-1)
println("relative residual norm = ", norm(b - A*x) / norm(b))

## 4.5 【発展】クリロフ部分空間法

### 4.5.5 Juliaの関数

In [ ]:
using Krylov                 # 事前にインストールが必要

In [ ]:
n = 1000
A = spdiagm(0 => range(2,2.5,n), 1 => ones(n-1), -1 => ones(n-1))
b = ones(n);

In [ ]:
x, status = cg(A,b)          # CG法の実行
res = norm(b - A * x) / norm(b)
println("Iter = ", status.niter, ", relative residual norm = ", res)

In [ ]:
using Random
Random.seed!(1234)
n = 1000; A = sprand(n,n,0.01) + 2 * spdiagm(ones(n)); b = randn(n);

In [ ]:
x, status = bicgstab(A,b)    # BiCGSTAB法の実行
res = norm(b - A * x) / norm(b)
println("Iter = ", status.niter, ", relative residual norm = ", res)